In [ ]:
import pandas as pd
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Get paths for data

train_data_path = '/content/drive/My Drive/food-allergy-identification/dataset/train'

test_data_path = '/content/drive/My Drive/food-allergy-identification/dataset/test'

In [ ]:
# Load train and test annotations CSV files
train_annotations = pd.read_csv('/content/drive/My Drive/food-allergy-identification/dataset/train-annotations.csv')
test_annotations = pd.read_csv('/content/drive/My Drive/food-allergy-identification/dataset/test-annotations.csv')

In [ ]:
def load_and_preprocess_images(filenames, folder_path):
    images = []
    for filename in filenames:
        image = cv2.imread(folder_path + filename)
        image = cv2.resize(image, (224, 224))
        images.append(image)
    return np.array(images)

In [ ]:
# Encode class labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)
num_classes = len(label_encoder.classes_)

NameError: name 'train_labels' is not defined

In [ ]:
# Define CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer=Adam(),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Evaluate the model

test_loss, test_acc = model.evaluate(test_images, test_labels_encoded)
print('Test accuracy:', test_acc)

# Make predictions on test data
predictions = model.predict(test_images)
predicted_labels = np.argmax(predictions, axis=1)

# Compute confusion matrix
conf_matrix = confusion_matrix(test_labels_encoded, predicted_labels)

# Plot confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
import pandas as pd
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import os  # Import os module to work with file paths

from google.colab import drive
drive.mount('/content/drive')

# Load the CSV file containing class labels
test_annotations = pd.read_csv('/content/drive/My Drive/food-allergy-identification/dataset/test-annotations.csv')

# Extract filenames and corresponding class labels
filenames = test_annotations['filename']
class_labels = test_annotations['class']

# Define a function to load and preprocess images
def load_and_preprocess_images(filenames, folder_path, class_labels):
    images = []
    labels = []
    for filename, class_label in zip(filenames, class_labels):
        # Construct the full path to the image file
        file_path = os.path.join(folder_path, filename)
        # Read the image using OpenCV
        image = cv2.imread(file_path)
        if image is None:
            print("Failed to read image:", file_path)
            continue
        # Resize the image to match the expected input shape of the model
        image = cv2.resize(image, (224, 224))
        # Optionally preprocess the image here
        images.append(image)
        labels.append(class_label)
    return np.array(images), np.array(labels)

# Load and preprocess test images
test_data_path = '/content/drive/My Drive/food-allergy-identification/dataset/test/'
test_images, test_labels = load_and_preprocess_images(filenames, test_data_path, class_labels)

# Ensure labels are properly encoded
label_encoder = LabelEncoder()
test_labels_encoded = label_encoder.fit_transform(test_labels)

# Define CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')  # Binary classification, so using sigmoid activation
])

# Compile the model
model.compile(optimizer=Adam(),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Evaluate the model
test_loss, test_acc = model.evaluate(test_images, test_labels_encoded)
print('Test accuracy:', 1 - test_acc)

# Make predictions
predictions = model.predict(test_images)
predicted_labels = np.round(predictions).flatten().astype(int)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
62/62 [==============================] - 46s 743ms/step - loss: -255.2208 - accuracy: 0.0189
Test accuracy: 0.9811416920274496
62/62 [==============================] - 53s 855ms/step


TypeError: Cannot cast array data from dtype('<U23') to dtype('int64') according to the rule 'safe'